# file_manager
This file manager handles the following tasks:
* fixes original dengue.json (parsing issue)
* Filters data and converts JSON to MongoDB

In [1]:
import pandas as pd
import pickle
import datetime
from datetime import timedelta, date
import re

import json
import ijson
import bz2
import os
from pprint import pprint
from IPython.display import clear_output

refs for dealing with large json:

http://www.aylakhan.tech/?p=27

https://stackoverflow.com/questions/10382253/reading-rather-large-json-files-in-python

https://stackoverflow.com/questions/2400643/is-there-a-memory-efficient-and-fast-way-to-load-big-json-files-in-python

# UFMG complete json of tweets

In [32]:
outer_path = r"D:/pseudo-dropbox/databases/ufmg/"
path = '../../inputs/'
file = os.path.join(outer_path,"dengue.json")
# file_fixed = os.path.join(outer_path,"dengue_fixed.json.bz2")
file_sample = os.path.join(outer_path,"dengue_sample.json")
outputs = '../outputs/'

# rewrite json sample with corrected parsing
Comma was missing from list of objects. So parsing was impossible.

JSON file has 7503436 objects.

In [28]:
def remove_special_char(text):
    text = re.sub('[áàãâ]', 'a', text)
    text = re.sub('[óòõô]', 'o', text)
    text = re.sub('[éèê]', 'e', text)
    text = re.sub('[íì]', 'i', text)
    text = re.sub('[úù]', 'u', text)
    text = re.sub('ç', 'c', text)
    text = re.sub('[ÁÀÃÂ]', 'A', text)
    text = re.sub('[ÓÒÕÔ]', 'O', text)
    text = re.sub('[ÉÈÊ]', 'E', text)
    text = re.sub('[ÍÌ]', 'I', text)
    text = re.sub('[ÚÙ]', 'U', text)
    text = re.sub('Ç', 'C', text)
    return text

In [34]:
file_len = 7503436
f_new = open(file_sample, "w")
f_new.write("[")
f_new.close()

with open(file, 'r', encoding="utf8") as f:
    count = 0
    f_new = open(file_sample, "a")
    for l in f:
        l = l.strip()
        l = remove_special_char(l)
        l = l.encode('ascii', errors='ignore').decode('ascii')
        
        if count == 0:
            f_new.write(l)
        elif count != 0:
            f_new.write(","+l)
        
        count += 1
        if count == 20: break

f_new.write("]")
f_new.close()

# see if tweets are ordered by date

In [35]:
test_list = []
with open(file_sample, 'r') as input_file:
    dates = ijson.items(input_file, 'item.created_at.$date')
    count = 0
    for date in dates:
        test_list.append(date)
        count += 1
        if count == 20: break
print(test_list)
all(test_list[i] <= test_list[i+1] for i in range(len(test_list)-1))

[1290197084000, 1290197105000, 1290197130000, 1290197158000, 1290197178000, 1290197190000, 1290197225000, 1290197238000, 1290197258000, 1290197331000, 1290197358000, 1290197482000, 1290197496000, 1290197599000, 1290197665000, 1290197703000, 1290197863000, 1290197927000, 1290197951000, 1290198007000]


True

Yes, dates seem to be ordered

### Get object structure

In [36]:
with open(file_sample, 'r') as input_file:
    items = ijson.items(input_file, 'item')
    count = 0
    for item in items:
        pprint(item)
        count += 1
        break

{'_id': 5713104428072960,
 'contributors': None,
 'control': {'classification': {'c': 'lac',
                                'd': 1443198861070,
                                'exp_dengue': {'campanha': Decimal('0.254248'),
                                               'exp_pessoal': Decimal('0.08588'),
                                               'informacao': Decimal('0.420341'),
                                               'opiniao': Decimal('0.101521'),
                                               'parodia': Decimal('0.13801'),
                                               'result': ['3'],
                                               'rules': 31},
                                'exp_dengue_bin': {'0': Decimal('0.905945'),
                                                   '1': Decimal('0.094055'),
                                                   'result': [False],
                                                   'rules': 11},
                                'spam_de

# save corrected and filtered data to MongoDB

# configure MongoDB

In [4]:
import pymongo
from pymongo import MongoClient
import dns
client = MongoClient()

In [6]:
db = client.twitter_fixed
collection_ufmg = db.ufmg_filtered
collection_ufmg_errors = db.ufmg_filtered_errors

To drop collection: 
```python
collection.drop()
```

# Filter JSON
Get most important features

In [68]:
collection_ufmg.drop()
collection_ufmg_errors.drop()

In [69]:
file_len = 7503436

with open(file, 'r', encoding="utf8") as f:
    count = 0
    for item in f:
        item = item.strip()
        item = re.sub('\$', '', item)
        item = json.loads(item)
        try:
            filtered_dict = {}
            filtered_dict["_id"] = item["_id"]
            if "date" in item["created_at"]: 
                filtered_dict["date"] = item["created_at"]["date"]
            else: filtered_dict["date"] = item["created_at"]
            if "lang" in item["control"]:
                if "lang" in item["control"]["lang"]:
                    filtered_dict["lang"] = item["control"]["lang"]["lang"]
                else: filtered_dict["lang"] = item["control"]["lang"]

            filtered_dict["text"] = item["text"]
            if "extended_tweet" in item: filtered_dict["extended_tweet"] = item["extended_tweet"]
            filtered_dict["screen_name"] = item["user"]["screen_name"]
            filtered_dict["user_id"] = item["user"]["id"]
            if item["coordinates"]: 
                coordinates = item["coordinates"]["coordinates"]
                filtered_dict["coordinates"] = [float(i) for i in coordinates]
            if item["place"]: filtered_dict["place"] = item["place"]["full_name"]
            if item["user"]["location"]: filtered_dict["location"] = item["user"]["location"]
            if "classification" in item["control"]: 
                if "campanha"    in item["control"]["classification"]["exp_dengue"]: filtered_dict["class_campanha"]    = float(item["control"]["classification"]["exp_dengue"]["campanha"])
                if "exp_pessoal" in item["control"]["classification"]["exp_dengue"]: filtered_dict["class_exp_pessoal"] = float(item["control"]["classification"]["exp_dengue"]["exp_pessoal"]) 
                if "informacao"  in item["control"]["classification"]["exp_dengue"]: filtered_dict["class_informacao"]  = float(item["control"]["classification"]["exp_dengue"]["informacao"]) 
                if "opiniao"     in item["control"]["classification"]["exp_dengue"]: filtered_dict["class_opiniao"]     = float(item["control"]["classification"]["exp_dengue"]["opiniao"]) 
                if "parodia"     in item["control"]["classification"]["exp_dengue"]: filtered_dict["class_parodia"]     = float(item["control"]["classification"]["exp_dengue"]["parodia"]) 
            collection_ufmg.insert_one(filtered_dict)
        except:
            print(count)
            print(item)
            collection_ufmg_errors.insert_one(item)
        
        count += 1
        if count % (int(file_len/1000)) == 0:
            clear_output()
            frac = count/file_len*100
            print("%.1f" % frac, "% done", sep="")

100.0% done


Dumping Mongo Database:

`mongodump --db DataBaseName --collection collection -o "path/to/folder"`

Copy the generated dump/DataBaseName folder to the new machine. Then, import using mongorestore:

`mongorestore --db DataBaseName /path/to/DataBaseName`
* the dump is the bson unzipped file

Note that /path/to/DataBaseName should be a directory filled with .json and .bson representations of your data.

You can also compress the file on the fly:

`mongodump --db somedb --collection somecollection --out "path/to/folder" --gzip`

# check for specific objects
Some tweets have incomplete text. So we need to check if they had extended tweet feature.

In [8]:
file_len = 7503436
cut_tweets = []
with bz2.open(file_fixed, 'r') as input_file:
    count = 0
    count_group = 0
    items = ijson.items(input_file, 'item')
    
    for item in items:
        if item["_id"] in [998292557569413120, 857264195452186625, 700288316114132992, 699922663909826560, \
                              996725579813711872, 707682665151987712, 820659778275995649, 716645598313754625, \
                              700284781658054656, 705010688704552960, 705874950327037952, 718277523306594305]:
            cut_tweets.append(item)
            count_group += 1
            if count_group == 10: break
        count += 1

        if count % (int(file_len/1000)) == 0:
            clear_output()
            frac = count/file_len*100
            print("%.1f" % frac, "% done", sep="")
print(cut_tweets)

91.5% done
[{'_id': 699922663909826560, 'favorited': False, 'contributors': None, 'truncated': False, 'text': 'RT @sergueirock: Precisamos bater mais palmas pro Sol. Alem de ser cosmicamente saudavel, e um ato que pode esmagar sem querer alguns mosqu', 'is_quote_status': False, 'in_reply_to_status_id': None, 'user': {'follow_request_sent': None, 'profile_use_background_image': True, 'default_profile_image': False, 'id': 290284050, 'verified': False, 'profile_image_url_https': 'https://pbs.twimg.com/profile_images/666718312903127040/6VPiF1ll_normal.jpg', 'profile_sidebar_fill_color': 'F6F6F6', 'profile_text_color': '333333', 'followers_count': 55, 'profile_sidebar_border_color': 'EEEEEE', 'id_str': '290284050', 'profile_background_color': 'ACDED6', 'listed_count': 0, 'profile_background_image_url_https': 'https://abs.twimg.com/images/themes/theme18/bg.gif', 'utc_offset': -7200, 'statuses_count': 687, 'description': '???', 'friends_count': 44, 'location': None, 'profile_link_color': '038

#### the samples were stored at '../outputs/tweet_check.json', but not properly formated as JSON. The file shows that those incomplete texts don't have the extended_tweet version neither. What should we do then???